In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader



In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

train_transform = transforms.Compose([
    transforms.RandomCrop(32, 4),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.CIFAR10("./data", True, train_transform, download=False)
test_dataset = torchvision.datasets.CIFAR10("./data", False, test_transform, download=False)

train_loader = torch.utils.data.DataLoader(train_dataset, 64, True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, 64, True, num_workers=2)

classes = train_dataset.classes

In [3]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(64*8*8, 512)
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
    
model = SimpleCNN()
x = torch.randn(64, 3, 32, 32)
model(x).size()

torch.Size([64, 10])

In [4]:
# from torchvision.models import resnet18, ResNet18_Weights

# model = resnet18(ResNet18_Weights.DEFAULT)

# for param in model.parameters():
#     param.requires_grad() = False
    
# num_features = model.fc.in_features
# model.fc = nn.Linear(num_features, 10)

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [6]:
def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        for data in train_loader:
            X, y = data
            X, y = X.to(device), y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            
            loss = criterion(output, y)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            
        print(f"epoch:{epoch+1}, loss : {running_loss / len(train_loader)}")
            
train(model, train_loader, criterion, optimizer, 5)
            

epoch:1, loss : 1.7784260958052047
epoch:2, loss : 1.5587548331531418
epoch:3, loss : 1.462703812610158
epoch:4, loss : 1.4008450547752478
epoch:5, loss : 1.3561384155012457


In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

def evaluate(model, test_loader):
    model.eval() # 모델을 평가 모드로 설정
    all_preds = []
    all_labels = []

    # 평가 시에는 기울기 계산이 필요 없으므로 no_grad() 사용
    with torch.no_grad():
        for data in test_loader:
            X, y = data
            X, y = X.to(device), y.to(device)
            
            outputs = model(X)
            _, predicted = torch.max(outputs.data, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
    
    # 1. 평가 지표 계산
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    
    
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    
evaluate(model, test_loader)

Accuracy: 0.5579
Precision: 0.5572
Recall: 0.5579
F1 Score: 0.5464
